In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Basic DIY ReAct Agent

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_diy_react_agent.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Ffunction-calling%2Fintro_diy_react_agent.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_diy_react_agent.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/function-calling/intro_diy_react_agent.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

| | |
|-|-|
|Author(s) | [Gary Ng](https://github.com/gkcng) |

## Overview

This notebook illustrates that at its simplest, a ReAct agent is a piece of code that coordinates between reasoning and acting, where:
- The reasoning is carried out by the language model
- The application code performs the acting, at the instruction of the language model.

<div>
    <table align="center">
      <tr><td>
        <img src="https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEiuuYg9Pduep9GkUfjloNVOiy3qjpPbT017GKlgGEGMaLNu_TCheEeJ7r8Qok6-0BK3KMfLvsN2vSgFQ8xOvnHM9CAb4Ix4I62bcN2oXFWfqAJzGAGbVqbeCyVktu3h9Dyf5ameRe54LEr32Emp0nG52iofpNOTXCxMY12K7fvmDZNPPmfJaT5zo1OBQA/s595/Screen%20Shot%202022-11-08%20at%208.53.49%20AM.png" alt="The Reasoning and Acting Cycle" width="500" align="center"/>
      </td></tr>
      <tr><td><div align="center"><em>From the paper: <a href="https://research.google/blog/react-synergizing-reasoning-and-acting-in-language-models/">ReAct: Synergizing Reasoning and Acting in Language Models</a></em></div></td></tr>
    </table>
</div>

This allows problems to be solved by letting a model 'think' through the tasks step-by-step, taking actions and getting action feedback before determining the next steps.

### Objectives

This notebook illustrates building the same agent in three different ways:

1. Use the Gemini single turn `generate_content` API. This also illustrates the concept of explicit goal checking vs model-based goal checking
1. Use the Gemini `ChatSession` API instead
1. Modified from raw prompting to use Gemini Function Calling

This example was suggested by Gemini Advanced as a simple, text-based demo that highlights the core ReAct concept: Autonomy, Cyclic, Reasoning. The agent's thoughts demonstrate a simple form of reasoning, connecting observations to actions.

<div>
    <table align="center">
      <tr><td>
        <img src="https://services.google.com/fh/files/misc/gemini_react_suggestion.jpg" alt="Gemini's suggestion" width="500" align="center"/>
      </td></tr>
      <tr><td><div align="center"><em>Scenario: A ReAct agent designed to tidy up a virtual room.</em></div></td></tr>
    </table>
</div>

### Costs

This tutorial uses billable components of Google Cloud:

- Google Foundational Models on Vertex AI ([Function Calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling#pricing))

Learn about [Generative AI on Vertex AI Pricing](https://cloud.google.com/vertex-ai/generative-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Install Vertex AI SDK for Python
This notebook uses the [Vertex AI SDK for Python](https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest).

In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [1]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Imports Libraries

In [3]:
import json
import sys
import traceback
from typing import Callable, Tuple

from google.protobuf.json_format import MessageToJson
from vertexai import generative_models
from vertexai.generative_models import (FunctionDeclaration, GenerativeModel,
                                        Part, Tool)

### Prepare a model with system instructions

In [4]:
model = GenerativeModel(
    "gemini-1.5-pro-preview-0514",
    system_instruction=[
        "You are an assistant that helps me tidy my room."
        "Your goal is to make sure all the books are on the shelf, all clothes are in the hamper, and the trash is empty.",
        "You cannot receive any input from me.",
    ],
    generation_config={"temperature": 0.2},
    safety_settings=[
        generative_models.SafetySetting(
            category=generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            method=generative_models.SafetySetting.HarmBlockMethod.PROBABILITY,
            threshold=generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        ),
    ],
)

## Helper Functions

In [5]:
verbose = True

In [6]:
# Conveience function to print multiline text indented
def indent(text, amount, ch=" "):
    padding = amount * ch
    return "".join(padding + line for line in text.splitlines(True))


# Convenience function for logging statements
def logging(msg):
    global verbose
    print(msg) if verbose else None


# Retrieve the text from a model response
def get_text(resp):
    return resp.candidates[0].content.parts[0].text


# Retrieve the function call information from a model response
def get_function(resp):
    return resp.candidates[0].content.parts[0].function_call


def get_action_label(json_payload, log, role="MODEL"):
    log(f"{role}: {json_payload}")
    answer = json.loads(json_payload)
    action = answer["next_action"]
    # rationale = answer["rationale"]
    return action


def get_action_from_func(func_payload, log, role="MODEL"):
    json_payload = MessageToJson(func_payload._pb)
    log(f"{role}: {json_payload}")
    return func_payload.name

### Action definitions
These are the pseudo actions declared as simple python functions. With the Function Calling pattern, the orchestration layer of an agent will be calling these Tools to carry out actions.

In [7]:
# Initial room state
def reset_room_state(room_state):
    room_state.clear()
    room_state["clothes"] = "floor"
    room_state["books"] = "scattered"
    room_state["wastebin"] = "empty"


# Functions for actions (replace these with Gemini function calls)
def pick_up_clothes(room_state):
    room_state["clothes"] = "carrying by hand"
    return room_state, "The clothes are now being carried."


def put_clothes_in_hamper(room_state):
    room_state["clothes"] = "hamper"
    return room_state, "The clothes are now in the hamper."


def pick_up_books(room_state):
    room_state["books"] = "in hand"
    return room_state, "The books are now in my hand."


def place_books_on_shelf(room_state):
    room_state["books"] = "shelf"
    return room_state, "The books are now on the shelf."


def empty_wastebin(room_state):
    room_state["wastebin"] = "empty"
    return room_state, "The wastebin is emptied."


# Maps a function string to its respective function reference.
def get_func(action_label):
    return None if action_label == "" else getattr(sys.modules[__name__], action_label)

### Explicit goals checking
This is only used in the first example to illustrate the concept: The goal checking responsibility can be either in code or be delegated to the model, depending on factors such as the complexity of the goal, ease of defining in code for example.

In [8]:
# Function to check if the room is tidy
# Some examples below do not call this function,
# for those examples the model takes on the goal validation role.
def is_room_tidy(room_state):
    return all(
        [
            room_state["clothes"] == "hamper",
            room_state["books"] == "shelf",
            room_state["wastebin"] == "empty",
        ]
    )

### Prompt Templates

In [9]:
functions = """
<actions>
    put_clothes_in_hamper - place clothes into hamper, instead of carrying them around in your hand.
    pick_up_clothes - pick clothes up from the floor.
    pick_up_books - pick books up from anywhere not on the shelf
    place_books_on_shelf - self explanatory.
    empty_wastebin - self explanatory.
    done - when everything are in the right place.
</actions>"""


def get_next_step_full_prompt(state, cycle, log):
    observation = f"The room is currently in this state: {state}."
    prompt = "\n".join(
        [
            observation,
            f"You can pick any of the following action labels: {functions}",
            "Which one should be the next step to achieve the goal? ",
            'Return a single JSON object containing fields "next_action" and "rationale".',
        ]
    )
    log("PROMPT:\n{}".format(indent(prompt, 1, "\t"))) if cycle == 1 else log(
        f"OBSERVATION: {observation}"
    )

    return prompt

## Example 1: Multiple single-turn `generate_content` calls with full prompts

An example turn.

```
You are an assistant that helps me tidy my room.
Your goal is to make sure all the books are on the shelf, all clothes are in the hamper, and the trash is empty.
You cannot receive any input from me.

The room is currently in this state: {'clothes': 'floor', 'books': 'scattered', 'wastebin': 'empty'}.

You can pick any of the following action labels:
<actions>
    put_clothes_in_hamper - place clothes into hamper, instead of carrying them around in your hand.
    pick_up_clothes - pick clothes up from the floor.
    pick_up_books - pick books up from anywhere not on the shelf
    place_books_on_shelf - self explanatory.
    empty_wastebin - self explanatory.
    done - when everything are in the right place.
</actions>
Which one should be the next step to achieve the goal?
Return a single JSON object containing fields "next_action" and "rationale".

RAW MODEL RESPONSE:

candidates {
  content {
    role: "model"
    parts {
      text: "{\"next_action\": \"pick_up_clothes\", \"rationale\": \"The clothes are on the floor and need to be picked up before they can be put in the hamper.\"}\n"
    }
  }
  finish_reason: STOP,
  ...
}
```

### The Main ReAct Loop
Interleaving asking for next steps and executing the steps

Notice that at cycle 4 the environment has changed to have an non-empty wastebin.
With the goal that includes trash being empty, the model is recognizing the change and behave according without need to respecify anything.

This is also well within expectation as this prompts the model fully everytime.

In [10]:
# Main ReAct loop
def main_react_loop(loop_continues, log):
    room_state = {}
    reset_room_state(room_state)
    trash_added = False

    cycle = 1
    while loop_continues(cycle, room_state):

        log(f"Cycle #{cycle}")

        # Observe the environment (use Gemini to generate an action thought)
        try:  # REASON #

            response = model.generate_content(
                get_next_step_full_prompt(room_state, cycle, log),
                generation_config={"response_mime_type": "application/json"},
            )  # JSON Mode
            action_label = get_action_label(get_text(response).strip(), log)

        except Exception:
            traceback.print_exc()
            log(response)
            break

        # Execute the action and get the observation
        if action_label == "done":
            break

        try:  # ACTION #

            # Call the function mapped from the label
            room_state, acknowledgement = get_func(action_label)(room_state)
            log(f"ACTION:   {action_label}\nEXECUTED: {acknowledgement}\n")

        except Exception:
            log("No action suggested.")

        # Simulating a change in environment
        if cycle == 4 and not trash_added:
            room_state["wastebin"] = "1 item"
            trash_added = True

        cycle += 1
        # End of while loop

    # Determine the final result
    result = (
        "The room is tidy!" if is_room_tidy(room_state) else "The room is not tidy!"
    )

    return room_state, result

In [11]:
# We are passing in a while loop continuation test function:
# Continue while loop when number of cycles <= 10 AND the room is not yet tidy.
# We are explicitly testing if the room is tidy within code.
#
# To save space, only the first cycle prints the full prompt.
# The same prompt template is used for every model call with a modified room state.
room_state, result = main_react_loop(
    lambda c, r: c <= 10 and not is_room_tidy(r), logging
)
print(room_state, result)

Cycle #1
PROMPT:
	The room is currently in this state: {'clothes': 'floor', 'books': 'scattered', 'wastebin': 'empty'}.
	You can pick any of the following action labels: 
	<actions>
	    put_clothes_in_hamper - place clothes into hamper, instead of carrying them around in your hand.
	    pick_up_clothes - pick clothes up from the floor.
	    pick_up_books - pick books up from anywhere not on the shelf
	    place_books_on_shelf - self explanatory.
	    empty_wastebin - self explanatory.
	    done - when everything are in the right place.
	</actions>
	Which one should be the next step to achieve the goal? 
	Return a single JSON object containing fields "next_action" and "rationale".
MODEL: {"next_action": "pick_up_clothes", "rationale": "The clothes are on the floor and need to be picked up before they can be put in the hamper."}
ACTION:   pick_up_clothes
EXECUTED: The clothes are now being carried.

Cycle #2
OBSERVATION: The room is currently in this state: {'clothes': 'carrying by hand

### The Model decides when the goal is reached

The model can also decide if the goal has been reached, instead of the application explicitly testing for the condition.
This is useful in scenarios where the goal state is variable and/or too complex to define in code.

To facilitate that, 

Instead of:

    while cycle <= 10 and not is_room_tidy(room_state):

We just have

    while cycle <= 10:
    
Remember we have previously defined an action "done" above, even though it is not a real function,
the model and the application can utilize that to determine termination. Note this creates an extra cycle.


In [12]:
# We are passing in a while loop continuation test function:
# Continue while loop when number of cycles <= 10
# We are no longer testing if the room is tidy within code.
# The decision is now up to the model.
room_state, result = main_react_loop(lambda c, r: c <= 10, logging)
print(room_state, result)

Cycle #1
PROMPT:
	The room is currently in this state: {'clothes': 'floor', 'books': 'scattered', 'wastebin': 'empty'}.
	You can pick any of the following action labels: 
	<actions>
	    put_clothes_in_hamper - place clothes into hamper, instead of carrying them around in your hand.
	    pick_up_clothes - pick clothes up from the floor.
	    pick_up_books - pick books up from anywhere not on the shelf
	    place_books_on_shelf - self explanatory.
	    empty_wastebin - self explanatory.
	    done - when everything are in the right place.
	</actions>
	Which one should be the next step to achieve the goal? 
	Return a single JSON object containing fields "next_action" and "rationale".
MODEL: {"next_action": "pick_up_clothes", "rationale": "The clothes are on the floor and need to be picked up before they can be put in the hamper."}
ACTION:   pick_up_clothes
EXECUTED: The clothes are now being carried.

Cycle #2
OBSERVATION: The room is currently in this state: {'clothes': 'carrying by hand

## Example 2: Incremental Messaging Using the Chat API

### The Chat session loop

In [13]:
# Main ReAct loop
def main_react_loop_chat(session, loop_continues, log):
    room_state = {}
    reset_room_state(room_state)
    trash_added = False

    prev_action = None
    msg = ""
    cycle = 1
    while loop_continues(cycle, room_state):

        log(f"Cycle #{cycle}")
        # Observe the environment (use Gemini to generate an action thought)
        try:  # REASON #

            if prev_action:
                msg = "\n".join(
                    [
                        prev_action,
                        f"ENVIRONMENT: The room is currently in this state: {room_state}.",
                        "Which should be the next action?",
                    ]
                )
                log("MESSAGE:\n{}".format(indent(msg, 1, "\t")))
            else:
                msg = get_next_step_full_prompt(room_state, cycle, log)

            # MODEL CALL
            response = session.send_message(
                msg, generation_config={"response_mime_type": "application/json"}
            )
            action_label = get_action_label(get_text(response).strip(), log)

        except Exception:
            traceback.print_exc()
            log(response)
            break

        # Execute the action and get the observation
        if action_label == "done":
            break

        try:  # ACTION #

            # Call the function mapped from the label
            room_state, acknowledgement = get_func(action_label)(room_state)
            prev_action = f"ACTION:   {action_label}\nEXECUTED: {acknowledgement}\n"
            log(prev_action)

        except Exception:
            log("No action suggested.")

        # Simulating a change in environment
        if cycle == 4 and not trash_added:
            room_state["wastebin"] = "1 item"
            trash_added = True

        cycle += 1
        # End of while loop

    # Determine the final result
    result = (
        "The room is tidy!" if is_room_tidy(room_state) else "The room is not tidy!"
    )

    return room_state, result

In [14]:
session = model.start_chat()

room_state, result = main_react_loop_chat(session, lambda c, r: c <= 10, logging)
print(room_state, result)

Cycle #1
PROMPT:
	The room is currently in this state: {'clothes': 'floor', 'books': 'scattered', 'wastebin': 'empty'}.
	You can pick any of the following action labels: 
	<actions>
	    put_clothes_in_hamper - place clothes into hamper, instead of carrying them around in your hand.
	    pick_up_clothes - pick clothes up from the floor.
	    pick_up_books - pick books up from anywhere not on the shelf
	    place_books_on_shelf - self explanatory.
	    empty_wastebin - self explanatory.
	    done - when everything are in the right place.
	</actions>
	Which one should be the next step to achieve the goal? 
	Return a single JSON object containing fields "next_action" and "rationale".
MODEL: {"next_action": "pick_up_clothes", "rationale": "The clothes need to be picked up from the floor and put in the hamper."}
ACTION:   pick_up_clothes
EXECUTED: The clothes are now being carried.

Cycle #2
MESSAGE:
	ACTION:   pick_up_clothes
	EXECUTED: The clothes are now being carried.
	
	ENVIRONMENT: Th

### Display the full chat history

In [15]:
print(session.history)

[role: "user"
parts {
  text: "The room is currently in this state: {\'clothes\': \'floor\', \'books\': \'scattered\', \'wastebin\': \'empty\'}.\nYou can pick any of the following action labels: \n<actions>\n    put_clothes_in_hamper - place clothes into hamper, instead of carrying them around in your hand.\n    pick_up_clothes - pick clothes up from the floor.\n    pick_up_books - pick books up from anywhere not on the shelf\n    place_books_on_shelf - self explanatory.\n    empty_wastebin - self explanatory.\n    done - when everything are in the right place.\n</actions>\nWhich one should be the next step to achieve the goal? \nReturn a single JSON object containing fields \"next_action\" and \"rationale\"."
}
, role: "model"
parts {
  text: "{\"next_action\": \"pick_up_clothes\", \"rationale\": \"The clothes need to be picked up from the floor and put in the hamper.\"}\n"
}
, role: "user"
parts {
  text: "ACTION:   pick_up_clothes\nEXECUTED: The clothes are now being carried.\n\nENV

## Example 3: Leveraging Gemini Function Calling Support

For more details please refer to the documentation on [Function Calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling). 

In the last two examples we simulated the function calling feature by explicitly prompting the model with a list of action labels and setting a JSON mode output. This example uses the function calling feature.

### Tool Declarations

In [16]:
# Functions for actions (replace these with Gemini function calls)
pick_up_clothes_func = FunctionDeclaration(
    name="pick_up_clothes",
    description="The act of picking clothes up from any place",
    parameters={"type": "object"},
)

put_clothes_in_hamper_func = FunctionDeclaration(
    name="put_clothes_in_hamper",
    description="Put the clothes being carried into a hamper",
    parameters={"type": "object"},
)

pick_up_books_func = FunctionDeclaration(
    name="pick_up_books",
    description="The act of picking books up from any place",
    parameters={"type": "object"},
)

place_books_on_shelf_func = FunctionDeclaration(
    name="place_books_on_shelf",
    description="Put the books being carried onto a shelf",
    parameters={"type": "object"},
)

empty_wastebin_func = FunctionDeclaration(
    name="empty_wastebin",
    description="Empty out the wastebin",
    parameters={"type": "object"},
)

done_func = FunctionDeclaration(
    name="done", description="The goal has been reached", parameters={"type": "object"}
)

room_tools = Tool(
    function_declarations=[
        pick_up_clothes_func,
        put_clothes_in_hamper_func,
        pick_up_books_func,
        place_books_on_shelf_func,
        empty_wastebin_func,
        done_func,
    ],
)

### Model with tool declarations

In [17]:
# Tools can be passed in during the initial creation of the model reference, or during send_message(), and generate_content()
# The choice depends on the variability of the set of tools to be used.
#
# model_fc = GenerativeModel(
#     "gemini-1.5-pro-preview-0514",
#     system_instruction=[
#        "You are an assistant that helps me tidy my room."
#        "Your goal is to make sure all the books are on the shelf, all clothes are in the hamper, and the trash is empty.",
#        "You cannot receive any input from me."
#    ],
#    tools=[ room_tools ],
#  )

### The function calling model response
With Function Calling, the choices of the tools are supplied through the API and is no longer necessary to include them in your prompt, and also unnecessary to specify the output format. For more details see the function calling [API Reference](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/function-calling#python_1).
```
response = session.send_message( msgs, tools=[ room_tools ]) 
```

The following raw model response is expected:
```
MESSAGE:
	ENVIRONMENT: The room is currently in this state: {'clothes': 'floor', 'books': 'scattered', 'wastebin': 'empty'}.
	Which should be the next action?

RAW RESPONSE:

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "pick_up_clothes"
        args {
        }
      }
    }
  },
  finish_reason: STOP,
  ...
}
```

In [18]:
# Wrapping the observation and model calling code into a function for better main loop readability.
def observe_and_reason(session, state: dict, prev_action: str, log: Callable) -> str:
    """Uses the language model (Gemini) to select the next action."""
    try:
        msgs = []
        if prev_action:
            msgs.append(
                Part.from_function_response(
                    name="previous_action", response={"content": prev_action}
                )
            )

        prompt = "\n".join(
            [
                f"ENVIRONMENT: The room is currently in this state: {state}.",
                "Which should be the next action?",
            ]
        )
        msgs.append(prompt)
        log(
            "MESSAGE:\n{}".format(
                indent(
                    "\n".join([prev_action, prompt] if prev_action else [prompt]),
                    1,
                    "\t",
                )
            )
        )

        response = session.send_message(
            msgs, tools=[room_tools]
        )  # JSON mode unnecessary.
        action_label = get_action_from_func(get_function(response), log)
        return action_label

    except Exception:
        log(f"Error during action selection: {e}")
        traceback.print_exc()
        return "done"  # Or a suitable default action

In [21]:
# Wrapping the action execution code into a function for better main loop readability.
def execute_action(state: dict, action_label: str, log: Callable) -> Tuple[dict, str]:
    """Executes the action on the room state and returns the updated state and an acknowledgement."""
    try:
        # Call the function mapped from the label
        state, acknowledgement = get_func(action_label)(state)

    except Exception:
        acknowledgement = "No action suggested or action not recognized."

    return state, acknowledgement

In [22]:
# Main ReAct loop
def main_react_loop_chat_fc(session, loop_continues, log):
    room_state = {}
    reset_room_state(room_state)
    trash_added = False

    prev_action = None
    cycle = 1
    while loop_continues(cycle, room_state):

        log(f"Cycle #{cycle}")
        # Observe the environment (use Gemini to generate an action thought)
        action_label = observe_and_reason(session, room_state, prev_action, log)

        # Execute the action and get the observation
        if action_label == "done":
            break
        room_state, acknowledgement = execute_action(room_state, action_label, log)
        prev_action = f"ACTION:   {action_label}\nEXECUTED: {acknowledgement}"
        log(prev_action + "\n")

        # Simulating a change in environment
        if cycle == 4 and not trash_added:
            room_state["wastebin"] = "1 item"
            trash_added = True

        cycle += 1
        # End of while loop

    # Determine the final result
    result = (
        "The room is tidy!" if is_room_tidy(room_state) else "The room is not tidy!"
    )

    return room_state, result

In [23]:
session = model.start_chat()

room_state, result = main_react_loop_chat_fc(session, lambda c, r: c <= 10, logging)
print(room_state, result)

Cycle #1
MESSAGE:
	ENVIRONMENT: The room is currently in this state: {'clothes': 'floor', 'books': 'scattered', 'wastebin': 'empty'}.
	Which should be the next action?
MODEL: {
  "name": "pick_up_clothes",
  "args": {}
}
ACTION:   pick_up_clothes
EXECUTED: The clothes are now being carried.

Cycle #2
MESSAGE:
	ACTION:   pick_up_clothes
	EXECUTED: The clothes are now being carried.
	ENVIRONMENT: The room is currently in this state: {'clothes': 'carrying by hand', 'books': 'scattered', 'wastebin': 'empty'}.
	Which should be the next action?
MODEL: {
  "name": "put_clothes_in_hamper",
  "args": {}
}
ACTION:   put_clothes_in_hamper
EXECUTED: The clothes are now in the hamper.

Cycle #3
MESSAGE:
	ACTION:   put_clothes_in_hamper
	EXECUTED: The clothes are now in the hamper.
	ENVIRONMENT: The room is currently in this state: {'clothes': 'hamper', 'books': 'scattered', 'wastebin': 'empty'}.
	Which should be the next action?
MODEL: {
  "name": "pick_up_books",
  "args": {}
}
ACTION:   pick_up_b